<a href="https://colab.research.google.com/github/Klauszhao/GameCode/blob/master/inbalanceDataTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
# 连接 Google colab 的云盘，数据集存放在云盘中，如果你的数据集不在云盘中，这段代码可以注释掉，从自己本地读取数据集
from google.colab import drive
drive.mount('/content/gdrive')
!ls 'gdrive/My Drive/Data/creditCard'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
creditcard.csv


In [0]:
def RandomForestTest(x_train,y_train,x_test,y_test):  # 
    RF = RandomForestClassifier(n_jobs=3)
    # 训练样本
    RF.fit(x_train, y_train)

    y_pred_train = RF.predict(x_train)
    f1 = f1_score(y_train,y_pred_train)
    print("train F1 : {:.3f}".format(f1))
    
    if not( x_test is None) :
        y_pred_test = RF.predict(x_test)
        f1 = f1_score(y_test,y_pred_test)
        print("test F1 : {:.3f}".format(f1))
    
    
#     y_pred_test = RF.predict(x_test)
#     f1 = f1_score(x_test,y_pred_test)
#     print("test F1 : {:.3f}".format(f1))

In [0]:
dataFile="gdrive/My Drive/Data/creditCard/creditcard.csv"
data = pd.read_csv(dataFile , encoding='utf-8')

In [5]:
print("行数={},列数={}".format(data.shape[0],data.shape[1]))
print(data['Class'].value_counts())

行数=284807,列数=31
0    284315
1       492
Name: Class, dtype: int64


### 原始数据
用原始数据训练模型，并将训练集作为测试集，看下效果。

In [6]:
x = data.drop('Class', axis=1)
y = data['Class']

RandomForestTest(x,y,None,None)


train F1 : 0.978


### 下采样
针对样本数据多的类别，采取下采样的方法，数据量对齐样本数据少的类别数据，这两个样本用来训练模型，并预测全部数据集，虽然不合理，但是可以了解下采样的效果

In [8]:
def dataProcess(data):
    # Shuffle the Dataset.
    shuffled_df = data.sample(frac=1,random_state=4)

    # Put all the fraud class in a separate dataset.
    fraud_df = shuffled_df.loc[shuffled_df['Class'] == 1]

    #Randomly select 492 observations from the non-fraud (majority class)
    non_fraud_df=shuffled_df.loc[shuffled_df['Class']== 0].sample(n=492,random_state=42)

    # Concatenate both dataframes again
    normalized_df = pd.concat([fraud_df, non_fraud_df])
    print(normalized_df.shape)
    return normalized_df


under_sampling_data = dataProcess(data)
print("下采样数据类型：{}".format(under_sampling_data.shape))



x_nor = under_sampling_data.drop('Class', axis=1)
y_nor = under_sampling_data['Class']
RandomForestTest(x_nor,y_nor,x,y)


(984, 31)
下采样数据类型：(984, 31)
train F1 : 0.994
test F1 : 0.120


In [9]:
print(" - - - -  负采样后 - - - - ")

sm = SMOTE(sampling_strategy='minority', random_state=7)

oversampled_train_x,oversampled_train_y = sm.fit_sample(x,y)

print("上采样之后的数据集大小=",oversampled_train_x.shape)

RandomForestTest(oversampled_train_x,oversampled_train_y,x,y)

 - - - -  负采样后 - - - - 
上采样之后的数据集大小= (568630, 30)
train F1 : 1.000
test F1 : 0.997


### 结论
很明显，上采样之后样本扩大了一倍，(568630, 30)，效果提升非常明显，用上采样得到的数据训练模型之后对原始数据集进行预测，效果也是提升非常好的，如果采用下采样，训练模型会很好，但是对原始数据集的拟合效果非常差，不建议用下采样
